In [ ]:
'''
This is to change image generation approach:

1. Separate images and models should exist for:
 - intersections 
 - normal lane following
 - lane changes
'''

In [9]:

# THIS GETS ANGLES WRONG

import carla #the sim library itself
import time # to set a delay after each photo
import cv2 #to work with images from cameras
import numpy as np #in this example to change image representation - re-shaping
import math
import sys
import random
sys.path.append('C:/CARLA_0.9.13/PythonAPI/carla') # tweak to where you put carla
from agents.navigation.global_route_planner import GlobalRoutePlanner

NUM_CYCLES = 1  #number of routes to go through in each town

SHOW_RGB = False

CAMERA_POS_Z = 1.3 
CAMERA_POS_X = 1.4 

CAM_HEIGHT = 480
CAM_WIDTH = 640
FOV = 90 # field of view = focal length

YAW_ADJ_DEGREES = 15 #random spin angle max

def camera_callback(image,data_dict):
    data_dict['image'] = np.reshape(np.copy(image.raw_data),(image.height,image.width,4))[:, :, :3]
def sem_callback(image,data_dict):
    ########## IMPORTANT CHANGE for Semantic camera ##############
    image.convert(carla.ColorConverter.CityScapesPalette)
    data_dict['sem_image'] = np.reshape(np.copy(image.raw_data),(image.height,image.width,4))[:, :, :3]

def cleanup():
    for actor in world.get_actors().filter('*vehicle*'):
        actor.destroy()
    for actor in world.get_actors().filter('*sensor*'):
        actor.destroy()

# function to get angle between the car and target waypoint
def get_angle(car,wp):
    '''
    this function returns degrees between the car's direction 
    and direction to a selected waypoint
    '''
    vehicle_pos = car.get_transform()
    car_x = vehicle_pos.location.x
    car_y = vehicle_pos.location.y
    wp_x = wp.transform.location.x
    wp_y = wp.transform.location.y
    
    # vector to waypoint
    if ((wp_y - car_y)**2 + (wp_x - car_x)**2)**0.5 == 0:
        x=0
        y=0
    else:
        x = (wp_x - car_x)/((wp_y - car_y)**2 + (wp_x - car_x)**2)**0.5
        y = (wp_y - car_y)/((wp_y - car_y)**2 + (wp_x - car_x)**2)**0.5
        
    #car vector
    car_vector = vehicle_pos.get_forward_vector()
    degrees = math.degrees(np.arctan2(y, x) - np.arctan2(car_vector.y, car_vector.x))
    # extra checks on predicted angle when values close to 360 degrees are returned
    if degrees<-180:
        degrees = degrees + 360
    elif degrees > 180:
        degrees = degrees - 360
    return degrees


def draw_route(wp, route,seconds=5.0):
    #draw the next few points route in sim window - Note it does not
    # get into the camera of the car
    if len(route)-wp <25: # route within 25 points from end is red
        draw_colour = carla.Color(r=255, g=0, b=0)
    else:
        draw_colour = carla.Color(r=0, g=0, b=255)
    for i in range(50):
        if wp+i<len(route)-2:
            world.debug.draw_string(route[wp+i][0].transform.location, '^', draw_shadow=False,
                color=draw_colour, life_time=seconds,
                persistent_lines=True)
    return None


def select_random_route(position,locs):
    '''
    retruns a random route for the car/veh
    out of the list of possible locations locs
    where distance is longer than 100 waypoints
    '''    
    point_a = position.location #we start at where the car is or last waypoint
    sampling_resolution = 1
    grp = GlobalRoutePlanner(world.get_map(), sampling_resolution)
    # now let' pick the longest possible route
    min_distance = 100
    result_route = None
    route_list = []
    for loc in locs: # we start trying all spawn points 
                                #but we just exclude first at zero index
        cur_route = grp.trace_route(point_a, loc.location)
        if len(cur_route) > min_distance:
            route_list.append(cur_route)
    result_route = random.choice(route_list)
    return result_route

client = carla.Client('localhost', 2000)
time.sleep(5)
client.set_timeout(25)

towns = ['Town03','Town04','Town05','Town06','Town07','Town10']

for town in towns:

    client.load_world(town) 
    time.sleep(15)
    print('Loaded ',town)
    # get world and spawn points
    world = client.get_world()

    
    settings = world.get_settings()
    settings.synchronous_mode = False
    settings.no_rendering_mode = False
    if settings.synchronous_mode:
        settings.fixed_delta_seconds = 0.05
    world.apply_settings(settings)

    cleanup()
    spawn_points = world.get_map().get_spawn_points()

    for cycle_count in range(NUM_CYCLES):
        if cycle_count % 10 ==0:
            print('...doing route #',cycle_count+1,' of ',NUM_CYCLES,' in town ',town)
        start_point = random.choice(spawn_points)
        vehicle_bp = world.get_blueprint_library().filter('*model3*')
        vehicle = world.try_spawn_actor(vehicle_bp[0], start_point)
        time.sleep(2)
    
        #RGB CAM
        camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
        camera_bp.set_attribute('image_size_x', str(CAM_WIDTH)) # this ratio works in CARLA 9.14 on Windows
        camera_bp.set_attribute('image_size_y', str(CAM_HEIGHT))
        camera_bp.set_attribute('fov', str(FOV))
        camera_init_trans = carla.Transform(carla.Location(z=CAMERA_POS_Z,x=CAMERA_POS_X))
        #this creates the camera in the sim
        camera = world.spawn_actor(camera_bp,camera_init_trans,attach_to=vehicle)
        image_w = camera_bp.get_attribute('image_size_x').as_int()
        image_h = camera_bp.get_attribute('image_size_y').as_int()
        camera_data = {'image': np.zeros((image_h,image_w,3)),
                    'sem_image': np.zeros((image_h,image_w,3))}
        # this actually opens a live stream from the camera
        camera.listen(lambda image: camera_callback(image,camera_data))

        #Semantgic cam
        sem_camera_bp = world.get_blueprint_library().find('sensor.camera.semantic_segmentation')
        sem_camera_bp.set_attribute('image_size_x', str(CAM_WIDTH)) # this ratio works in CARLA 9.14 on Windows
        sem_camera_bp.set_attribute('image_size_y', str(CAM_HEIGHT))
        sem_camera_bp.set_attribute('fov', str(FOV))
        #this creates the camera in the sim
        sem_camera = world.spawn_actor(sem_camera_bp,camera_init_trans,attach_to=vehicle)
        image_w = sem_camera_bp.get_attribute('image_size_x').as_int()
        image_h = sem_camera_bp.get_attribute('image_size_y').as_int()
        sem_camera_data = {'image': np.zeros((image_h,image_w,3))}
        # this actually opens a live stream from the camera
        #sem_camera.listen(lambda image: camera_callback(image,sem_camera_data))
        sem_camera.listen(lambda image: sem_callback(image,camera_data))

        if SHOW_RGB:
            cv2.namedWindow('RGB Camera',cv2.WINDOW_AUTOSIZE)
            cv2.imshow('RGB Camera',camera_data['image'])
        #make a route
        route = select_random_route(start_point,spawn_points)
        gen_dir_angle = 0 # in case we did not get a general GPS direction
        for idx in range(len(route)-2):  # move the car through the route
            waypoint = route[idx]
            remaining_wp_count = len(route) - idx

            transform = waypoint[0].transform
            vehicle.set_transform(transform)
            vehicle.apply_control(carla.VehicleControl(throttle=0, steer=0, brake=1))
            time.sleep(2) #these delays seem to be necessary for teh car to take the position before a shot is taken
            initial_yaw = waypoint[0].transform.rotation.yaw
            
            # logic to detect a lane change to flag them
            draw_route(idx, route,seconds=5.0)
            
            lane_change = False
            
            if remaining_wp_count >2 and (route[idx+1][0].is_intersection or route[idx+1][0].is_junction):
                angle_wp30 = get_angle(vehicle,route[idx+30][0])
                if angle_wp30 > 15:
                    gen_dir_angle = 1
                elif angle_wp30 <-15:
                    gen_dir_angle = -1
                else:
                    gen_dir_angle = 0
            elif not waypoint[0].is_intersection and not waypoint[0].is_junction:
                if remaining_wp_count > 2:
                    if route[idx][0].lane_id != route[idx+1][0].lane_id:
                        lane_change = True
                if lane_change: # lane changes are treated as turns in general direction
                    if get_angle(vehicle,route[idx+1][0])<0:
                        gen_dir_angle = -1
                    else:
                        gen_dir_angle = 1
            else:
                gen_dir_angle =0

             # this is different from previous version - we will specify image type so we can put it into the name of file
            if remaining_wp_count >2 and route[idx+1][0].is_intersection or route[idx+1][0].is_junction:
                image_type = 'I' # I for intersection
            elif lane_change:
                image_type = 'L' # L for lane change
            else:
                image_type = 'N' # for Normal lane following

            # setting for intersections not to spin the car 
            if image_type !='I':
                for i in range(5):
                    trans = waypoint[0].transform
                    angle_adj = random.randrange(-YAW_ADJ_DEGREES, YAW_ADJ_DEGREES, 1)
                    if image_type == 'I':
                        angle_adj =0 # for intersections we remove the impact of a spin
                    trans.rotation.yaw = initial_yaw +angle_adj 
                    vehicle.set_transform(trans)
                    vehicle.apply_control(carla.VehicleControl(throttle=0, steer=0, brake=1))
                    time.sleep(1)  #these delays seem to be necessary for the car to take the position before a shot is taken
                    
                    # Display with imshow
                    if SHOW_RGB:
                        cv2.imshow('RGB Camera',camera_data['image'])
                    # save semantic image watching out for end of route    
                    if idx +5 < len(route)-1:
                        predicted_angle = get_angle(vehicle,route[idx+5][0]) # we always get the angle to +5 waypoint ahead of us
                        
                        time_grab = time.time_ns()
                        sem_image = camera_data['sem_image']
                        if np.sum(sem_image) > 0:   #check for black images
                            cv2.imwrite('_img_separated/%06d_%s_%s_%s.png' % (time_grab, gen_dir_angle,round(predicted_angle,0),image_type), sem_image)
        cleanup()

    cv2.destroyAllWindows()
    cleanup()

Loaded  Town03
Doing route # 0  of  1  in town  Town03


KeyboardInterrupt: 

In [8]:
# checking a bug in Town 3 when car was turning right but direction was -1
client = carla.Client('localhost', 2000)
time.sleep(5)

# get world and spawn points
world = client.get_world()

vehicle = world.get_actors().filter('*vehicle*')[0]


In [38]:
# script to incrementally change car's position to locate it where you need for debugging
old_pos = vehicle.get_transform()
new_pos = carla.Transform(old_pos.location + carla.Location(x=0,y=3.3),
                            carla.Rotation(yaw = old_pos.rotation.yaw + 0))
vehicle.set_transform(new_pos)

In [5]:
# select and draw a route quickly to confirm it goes where you want
def draw_all_route(route,seconds=5.0):
    #draw the next few points route in sim window - Note it does not
    # get into the camera of the car
    draw_colour = carla.Color(r=0, g=0, b=255)
    for i in range(len(route)-1):
        world.debug.draw_string(route[i][0].transform.location, '^', draw_shadow=False,
            color=draw_colour, life_time=seconds,
            persistent_lines=True)
    return None

#route = select_random_route(new_pos,world.get_map().get_spawn_points())
draw_all_route(route,seconds=5.0)

In [28]:
print(get_distant_angle(vehicle,0,route,30))

0


In [50]:
# debug get_distant_angle(car,wp_idx,rte, delta):
'''
This function determines general direction
given what we are planning to do at an oncoming intersection
so if we are within delta points from an intersection
we check our route at the intersection to decide what we do at it
'''

# set up fuction's vars
wp_idx = 0
delta =30
rte = route
#### end setup vars


if wp_idx + delta < len(rte)-1:
    i = wp_idx + delta
else:
    i = len(rte)-1
# check for intersection within the "look forward"
# so we do not give turn results when just following the road
intersection_detected = False
for x in range(i-wp_idx):
    if rte[wp_idx+x][0].is_junction:
            intersection_detected = True
            intersection_ref = wp_idx+x
            break
if not intersection_detected:
    result = 0
else: #we check out the intersection
    angles_planned = [] # this is a list of angles towards current exit from the intersection
    all_angles = []
    #this is how to get all waypoints defining an intersection
    junction_wps = rte[intersection_ref][0].get_junction().get_waypoints(carla.LaneType.Driving)
    for wp in junction_wps:
        angle = int(get_angle(vehicle,wp[1])) 
        #this 'if' below excludes close exits points to entry - exits for which you would need to take u-turns so they do not count
        if wp[1].transform.location.distance(route[intersection_ref][0].transform.location) > 20: 
            # check all exits for proximity to our route so we can flag the exit we are planning to take
            for i in range(intersection_ref,len(route)-1):
                if wp[1].transform.location.distance(route[i][0].transform.location) < 5:
                    angles_planned.append(angle)
                else:
                    all_angles.append(angle)
    angles_planned = list(set(angles_planned))
    all_angles = list(set(all_angles))
    alternative_angles = [item for item in all_angles if item not in angles_planned] 
    if len(alternative_angles) == 0 or len(angles_planned) == 0:
        result = 0
    elif min(angles_planned)<-25 and (min(alternative_angles) > min(angles_planned)):
        #we are planningleft turn
        result = -1
    elif max(angles_planned)>25 and (max(alternative_angles) < max(angles_planned)):
        result = 1
    else:
        result = 0  


In [51]:
print('result: ',result,
      ', intersection_detected: ', intersection_detected,
      ', route length i: ',i)

result:  0 , intersection_detected:  True , route length i:  915


In [47]:
draw_colour = carla.Color(r=0, g=0, b=255)
for wp in junction_wps:
    world.debug.draw_string(wp[1].transform.location, '^', draw_shadow=False,
        color=draw_colour, life_time=10,
        persistent_lines=True)


In [52]:
angles_planned

[]

In [49]:
all_angles

[-45, -49]

In [53]:
alternative_angles

[-45, -49]

In [56]:
print(get_angle(vehicle,route[40][0]))

59.964672919369455


In [59]:
world.debug.draw_string(route[30][0].transform.location, '^', draw_shadow=False,
    color=draw_colour, life_time=10,
    persistent_lines=True)

In [ ]:
''' 
outcome:
change intersection approach to getting overall direction angle on approach
keep that angle until off the intersection
maintain -1, 0, 1 space

'''